# Average Episode BPM
> Tracking average episode BPM over time.

- toc: true 
- badges: true
- comments: false
- categories: [asot]
- image: images/chart-preview.png

## Methodology

A quick call to the Spotify Web API can return over a dozen [audio features for a track](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/), notably `tempo` - "The overall estimated tempo of a track in beats per minute (BPM)."

Conveniently, the entire back catalogue of A State of Trance - 950+ episodes - has been uploaded to Spotify under the artist ["Armin van Buuren ASOT Radio"](https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce).

In [1]:
#hide
import os
import yaml
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

with open('spotipy_credentials.yaml', 'r') as spotipy_credentials_file:
    credentials = yaml.safe_load(spotipy_credentials_file)
    os.environ["SPOTIPY_CLIENT_ID"] = credentials['spotipy_credentials']['spotipy_client_id']
    os.environ["SPOTIPY_CLIENT_SECRET"] = credentials['spotipy_credentials']['spotipi_client_seret']

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

ASOT_SPOTIFY_ID = '25mFVpuABa9GkGcj9eOPce' # Armin van Buuren ASOT Radio (https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce)

episodes = []
results = sp.artist_albums(ASOT_SPOTIFY_ID, album_type='album')
episodes.extend(results['items'])
while results['next']:
    results = sp.next(results)
    episodes.extend(results['items'])
seen = set()  # to avoid dups
for episode in episodes:
    name = episode['name']
    if name not in seen:
        seen.add(name)

episodes = episodes[::-1]

In [2]:
#hide
episode_tracks = sp.album_tracks(episodes[1]['uri'])
print(json.dumps(episode_tracks, indent=2))

NameError: name 'json' is not defined

In [3]:
#hide
audio_features = sp.audio_features(episode_tracks['items'][2]['uri'])
print(json.dumps(audio_features, indent=2))

NameError: name 'json' is not defined

In [4]:
import altair as alt
import numpy as np
import pandas as pd

In [5]:
bpm = []
for track in episode_tracks['items']:
    bpm.append(sp.audio_features(track['uri'])[0]['tempo'])

x = np.arange(len(episode_tracks['items']))   

source = pd.DataFrame({
  'track': x,
  'bpm': np.array(bpm)
})

alt.Chart(source).mark_line().encode(
    x='track',
    y='bpm'
)

alt.Chart(...)

In [6]:
#hide
episode_avg_bpm = []

for episode in episodes[:10]:
    try:
        episode_bpm = 0
        tracks_counted = 0
        for track in sp.album_tracks(episode['uri'])['items']:
            if "A State Of Trance" in track['name']:
                continue
            else:
                episode_bpm += sp.audio_features(track['uri'])[0]['tempo']
                tracks_counted += 1
        avg = episode_bpm/tracks_counted
        print(episode['name'], avg)
        episode_avg_bpm.append(avg)
    except:
        pass

A State Of Trance Episode 001 138.86957142857142
A State Of Trance Episode 003 135.295
A State Of Trance Episode 004 137.3078
A State Of Trance Episode 005 137.122875
A State Of Trance Episode 007 137.459125
A State Of Trance Episode 008 136.409
A State Of Trance Episode 009 138.58033333333333
A State Of Trance Episode 010 137.5172
A State Of Trance Episode 012 138.5285
A State Of Trance Episode 015 138.09266666666667


In [7]:
x = np.arange(len(episode_avg_bpm))   

source = pd.DataFrame({
  'episode': x,
  'avg bpm': np.array(episode_avg_bpm)
})

alt.Chart(source).mark_line().encode(
    alt.X('episode'),
    alt.Y('avg bpm', scale=alt.Scale(domain=(120, 150))),
).properties(
    title="A State of Trance - Average BPM of episode"
)

alt.Chart(...)

In [8]:
episode_bpm = 0
tracks_counted = 0
for track in sp.album_tracks('56qM5Y21wbvCW9l5GiAiaV')['items']:
    if "A State Of Trance" in track['name']:
        continue
    else:
        print(track['name'], ' - ', sp.audio_features(track['uri'])[0]['tempo'])
        episode_bpm += sp.audio_features(track['uri'])[0]['tempo']
        tracks_counted += 1
print(episode_bpm/tracks_counted)

Cloudwalking [ASOT 057] - Beatpusher Remix  -  141.61
Ligaya [ASOT 057] - Original Mix  -  141.784
Obsession [ASOT 057] - Original Mix  -  136.377
Gatex [ASOT 057] - Original Mix  -  184.848
Positron [ASOT 057] - Marco V Remix  -  139.264
148.7766


In [9]:
x = np.arange(len(episode_avg_bpm))

source = pd.DataFrame({
  'episode': x,
  'avg bpm': np.array(episode_avg_bpm)
})

source['138'] = 138

base = alt.Chart(source).mark_line().encode(
    alt.X('episode'),
    alt.Y('avg bpm', scale=alt.Scale(domain=(120, 150))),
).properties(
    title="A State of Trance - Average BPM of episode"
)

rule = alt.Chart(source).mark_rule(color='red').encode(
    y='138'
)

base + rule

NameError: name 'line' is not defined